# Neural Network from Scratch
No libraries. 😎

WIP.

## Dense Layers
The most basic type of layer in artificial neural networks is the fully connected, or dense, layer in which every neuron in one layer is connected to every neuron in the next layer. Since this means we'll be taking the dot product of every input vector with every weight vector, this operation is indistinguishable from a matrix multiplication of the input as a row matrix and the weights as a column matrix.

In [1]:
abstract type Layer end

In [2]:
mutable struct DenseLayer <: Layer
    # In a dense layer, the neurons and biases are treated as column matrices
    neurons::Array
    bias::Array
    
    # Activation function for the layer and its first derivative
    ϕ::Function
    ∇ϕ::Function
    
    # Batch states for backprop
    input::Array   # Output of the previous layer
    net::Array     # input * neurons + bias
    output::Array  # activation(net)
      
    function DenseLayer(input_dim::Int, output_dim::Int, ϕ::Function)
        neurons = randn(input_dim, output_dim)
        bias = randn(1, output_dim)
        return new(neurons, bias, ϕ, gradient(ϕ))
    end
    
    function DenseLayer(neurons::Array{T}, bias::Array{T}, ϕ::Function) where T<:Real
        return new(neurons, bias, ϕ, gradient(ϕ))
    end

end;

### Dense Output Layers
The only real difference between a dense hidden layer and a dense output layer is that the output layer's neurons have a zero bias. They used to be separate structs but now they're not. :)

In [3]:
"""
Returns an instance of DenseLayer with zeroed out bias.
"""
function DenseOutputLayer(input_dim::Int, output_dim::Int, ϕ::Function)::DenseLayer
    L = DenseLayer(input_dim, output_dim, ϕ)
    L.bias = zeros(1, output_dim)
    return L
end

DenseOutputLayer

In [4]:
function batchnormalize(x::Array{T,N}, γ::T=1., β::T=0.)::Array{T,N} where T<:Real where N
    # μ = mean of x
    μ = sum(x)/length(x)
    # σ² = variance of x
    σ² = sum((x .- μ).^2)
    # Normalize x
    x̂ = (x .- μ) ./ sqrt(σ²)
    # Scale and shift
    y = x̂ .* γ .+ β
    return y
end;

In [5]:
function batchnormalize!(layer::T, γ=1., β=0.) where T<:Layer
    layer.output = batchnormalize(layer.output, γ, β)
end;

In [6]:
abstract type NeuralNetwork end
abstract type FeedForwardNeuralNetwork <: NeuralNetwork end

In [7]:
mutable struct FullyConnectedNeuralNetwork <: FeedForwardNeuralNetwork
    layers::Array{Layer}
    η::Float64
    
    loss::Function
    ∇loss::Function
    
    function FullyConnectedNeuralNetwork(input_dim::Int, hidden_dims::Vector{Int}, output_dim::Int, ϕ::Vector{Function}, loss::Function, η=0.01)
        layers = []
        
        push!(layers, DenseLayer(input_dim, hidden_dims[1], ϕ[1]))
        
        for i in 1:length(hidden_dims)-1
            push!(layers, DenseLayer(hidden_dims[i], hidden_dims[i+1], ϕ[i+1]))
        end
        
        push!(layers, DenseOutputLayer(hidden_dims[end], output_dim, ϕ[end]))
        
        return new(layers, η, loss, gradient(loss))
    end
    
    function FullyConnectedNeuralNetwork(layers::Vector{T}, loss::Function, η=0.01) where T<:Layer
        return new(layers, η, loss, gradient(loss))
    end
end;

In [8]:
function predict(model::FullyConnectedNeuralNetwork, data)
    for layer in model.layers
        data = data * layer.neurons .+ layer.bias
        data = layer.ϕ.(data)
    end
    return data[:,1]
end;

In [9]:
function train!(model::FullyConnectedNeuralNetwork, data, target, epochs=1, clear=true)
    for i in 1:epochs
        forwardpass!(model, data)
        backprop!(model, target)
    end
    if clear
        for i in 1:length(model.layers)
            model.layers[i].input = []
            model.layers[i].net = []
            model.layers[i].output = []
        end
    end
end;

In [10]:
function forwardpass!(model::FullyConnectedNeuralNetwork, data::Array{T}) where T<:Real
    for layer in model.layers 
        layer.input = data
        layer.net = data * layer.neurons .+ layer.bias
        layer.output = layer.ϕ.(layer.net)
        data = layer.output
    end
end;        

In [11]:
function backprop!(model::FullyConnectedNeuralNetwork, target::Array{T}) where T<:Real   
    # Backpropagate error by iteratively updating error delta terms δ
    # Work backwards from output layer
    layer = model.layers[end]
    # w:   weights
    # o:   output
    # net: output before activation
    # E:   error
    # Calculate partial derivative of error with respect to each weight
    # ∂E╱∂wᵢⱼ = ∂E╱∂oⱼ * ∂oⱼ╱∂netⱼ * ∂netⱼ╱∂wᵢⱼ
    # Partial derivative of loss
    𝜕𝐸╱𝜕𝑜 = model.∇loss.(layer.output, target)
    # Partial derivative of activated output
    𝜕𝑜╱𝜕𝑛𝑒𝑡 = layer.∇ϕ.(layer.net)
    # δ=∂E╱∂net
    # Error with respect to net -- the error terms
    𝛿 = 𝜕𝐸╱𝜕𝑜 .* 𝜕𝑜╱𝜕𝑛𝑒𝑡
    # ∂net╱∂w is equal to the transpose of the previous layer's output (https://bit.ly/backproperror)
    𝜕𝑛𝑒𝑡╱𝜕𝑤 = layer.input'
    # Calculate delta terms for the neurons and adjust by the learning rate
    𝜂 = model.η
    𝛥𝑤 = -𝜂 * 𝜕𝑛𝑒𝑡╱𝜕𝑤 * 𝛿
    # Update the weights of the output layer
    layer.neurons += 𝛥𝑤
    # Output layer has no bias, so no need to update it
    # Now do the rest of the layers in reverse order
    for L in length(model.layers)-1:-1:1
        layer = model.layers[L]
        # Need to calculate weight adjustment, Δwᴸ
        # Δwᴸ = -η * (oᴸ⁻¹)ᵀ * δᴸ
        # Make sure to save error terms δᴸ for backprop
        # δᴸ = δᴸ⁺¹ * (wᴸ⁺¹)ᵀ * ∇ϕᴸ(oᴸ⁻¹wᴸ)
        # Term oᴸ⁻¹wᴸ is layer L's unactivated output and stored as netᴸ
        # All together
        # Δwᴸ = -η * (oᴸ⁻¹)ᵀ * δᴸ⁺¹ * (wᴸ⁺¹)ᵀ * ∇ϕᴸ(oᴸ⁻¹wᴸ)
        𝜕𝐸╱𝜕𝑜 = 𝛿 * model.layers[L+1].neurons'
        𝜕𝑜╱𝜕𝑛𝑒𝑡 = layer.∇ϕ.(layer.net)
        𝛿 = 𝜕𝐸╱𝜕𝑜 .* 𝜕𝑜╱𝜕𝑛𝑒𝑡
        𝜕𝑛𝑒𝑡╱𝜕𝑤 = layer.input' 
        𝛥𝑤 = -𝜂 * 𝜕𝑛𝑒𝑡╱𝜕𝑤 * 𝛿
        # Update the neurons
        layer.neurons += 𝛥𝑤
        # Update the bias by adding scaled error terms
        layer.bias = layer.bias .+ (-𝜂 * 𝛿)
    end   
end; 

In [12]:
function fit!(model::FullyConnectedNeuralNetwork, data::Array{T}, target::Vector{T}, epochs::Int, verbose=false) where T<:Real    
    if verbose
        prediction = predict(model, data)
        loss = model.loss
        @show loss(prediction, target)
        print("Training for ", epochs, " epochs.")
        @time train!(model, data, target, epochs)
        prediction = predict(model, data)
        @show loss(prediction, target)
    else
        train!(model, data, target, epochs)
    end
end;

## Activation Functions
If a model is only comprised of linear operations such as matrix multiplication, it can only model linear phenomena. To get around this limitation we run the results of our linear operations through some non-linear operation, collectively referred to in this context as *activation functions* in reference to the activation threshold of biological neurons. Because for backprop we need to compute derivatives, it's generally advisable to go with activation functions that have easily computable derivatives.

### ReLU
A popular activation function for hidden layers is the Rectified Linear Unit, or ReLU, in which if the value is negative it's replaced with zero. This is clearly non-linear, trivial to compute, and has a derivative that's trivial to compute and can be predefined. This has some issues (though it still works in many cases), first and foremost being that all values below zero are collapsed to the same value. A minor tweak, named LeakyReLU, addresses this by dividing negative values by 100 instead of zeroing them out completely. Other variations exist that I'm not getting into here yet (PReLU, ELU, etc).

### Sigmoid
Explain the squishy of sigmoid

In [19]:
# Activations (ϕ)
function ReLU(x::T)::T where T<:Real 
    return max(x, 0)
end

function LeakyReLU(x::T)::T where T<:Real 
    return max(x, 0.01x)
end

function sigmoid(x::T)::T where T<:Real 
    return 1.0 / (1 + exp(-x))
end

# Derivatives (∇ϕ)
function ∇ReLU(x::T)::T where T<:Real 
    return x > 0
end

function ∇LeakyReLU(x::T)::T where T<:Real 
    return x < 0 ? 0.01 : 1.0
end

function ∇sigmoid(x::T)::T where T<:Real
    y = sigmoid(x)
    return y * (1 - y)
end

∇sigmoid (generic function with 1 method)

In [14]:
# Error Calculations

# Mean Squared Error
function mse(x::T, target::T)::T where T<:Real
    return .5(target-x)^2
end

function mse(xs::Vector{T}, target::T)::T where T<:Real 
    err(x) = target - x
    return sum(err.(xs).^2)/2*length(xs)
end

function mse(xs::Vector{T}, target::Vector{T})::T where T<:Real
    sum((xs .- target).^2)/2*length(xs)
end

# Derivatives
function ∇mse(x::T, target::T)::T where T<:Real
    return x - target
end

function ∇mse(xs::Vector{T}, target::T)::Vector{T} where T<:Real
    return xs .- target
end

function ∇mse(xs::Vector{T}, target::Vector{T})::Vector{T} where T<:Real
    return xs .- target
end;

In [15]:
function gradient(f::Function)::Function
    # Activation
    if f == ReLU
        ∇f = ∇ReLU
    elseif f == LeakyReLU
        ∇f = ∇LeakyReLU
    elseif f == sigmoid
        ∇f = ∇sigmoid
    elseif f == sin
        ∇f = cos
    
    # Loss
    elseif f == mse
        ∇f = ∇mse
    
    end
    
    return ∇f
end;

In [16]:
input_size = 4

Lᵢ = DenseLayer(input_size, 8, LeakyReLU)
L₂ = DenseLayer(8, 4, sigmoid)
L₃ = DenseLayer(4, 4, sigmoid)
Lₒ = DenseOutputLayer(4, 1, sigmoid)

Layers = [Lᵢ, L₂, L₃, Lₒ]
m = FullyConnectedNeuralNetwork(Layers, mse)

samples=4
data = randn(samples,input_size)
target = rand([0.,1.], samples)
fit!(m, data, target, 1000, true);

loss(prediction, target) = 2.1577619616481356
Training for 1000 epochs.  0.505927 seconds (1.85 M allocations: 96.296 MiB, 3.46% gc time)
loss(prediction, target) = 1.1576305697571925


In [17]:
# Can we overfit a disporportionately large model on a random matrix?
inputsize = 4
hidden_layers = [8,8,8,8,8,4,4,4,2,2,2,1]
output_size = 1
activations = vcat(repeat([LeakyReLU], length(hidden_layers)+1), sigmoid)
loss=mse;

model = FullyConnectedNeuralNetwork(inputsize, hidden_layers, output_size, activations, loss);

samples = 4
data = randn(samples, inputsize)
target = rand([0.,1.], samples)

fit!(model, data, target, 10000, true);

loss(prediction, target) = 2.2041308033103513
Training for 10000 epochs.  0.497362 seconds (3.21 M allocations: 365.298 MiB, 6.27% gc time)
loss(prediction, target) = 1.0029765415713756
